In [155]:
import numpy as np
import pandas as pd
import torch
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import torch.nn as nn

In [156]:
data = pd.read_csv("/Users/jesseseid/Jseid628/avocado_ripeness_dataset.csv")
data.head()

,firmness,hue,saturation,brightness,color_category,sound_db,weight_g,size_cm3,ripeness
0,14.5,19,40,26,black,34,175,261,ripe
1,71.7,53,69,75,green,69,206,185,pre-conditioned
2,88.5,60,94,46,dark green,79,220,143,hard
3,93.8,105,87,41,dark green,75,299,140,hard
4,42.5,303,58,32,purple,63,200,227,breaking


In [157]:
#Pre-processing

#Categorical to Numerical
le = LabelEncoder()
data['color_category'] = le.fit_transform(data['color_category'])

ripe_types = data['ripeness'].unique()
data['ripeness'].replace(ripe_types, list(range(0,5)), inplace=True)

#Scaling
scaler=MinMaxScaler()
column_list = list(data)
column_list.remove('ripeness')
for col in columns_list:
    reshaped = np.array(data[col]).reshape(-1,1)
    data[col] = scaler.fit_transform(reshaped)

#Features vs Labels
labels = data['ripeness']
del data['ripeness']
features = data
data.head()



,firmness,hue,saturation,brightness,color_category,sound_db,weight_g,size_cm3
0,0.047458,0.054878,0.144928,0.235294,0.000000,0.081633,0.156463,0.809045
1,0.693785,0.158537,0.565217,0.955882,0.666667,0.795918,0.367347,0.427136
2,0.883616,0.179878,0.927536,0.529412,0.333333,1.000000,0.462585,0.216080
3,0.943503,0.317073,0.826087,0.455882,0.333333,0.918367,1.000000,0.201005
4,0.363842,0.920732,0.405797,0.323529,1.000000,0.673469,0.326531,0.638191


In [158]:
#Split into test and train before conversion to tensors
features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size = 0.2)

#Convert features and labels to torch tensors
features_train_tensor = torch.tensor(features_train.values, dtype=torch.float32)
features_test_tensor = torch.tensor(features_test.values, dtype=torch.float32)

labels_train_tensor = torch.tensor(labels_train.values, dtype=torch.int64)
labels_test_tensor = torch.tensor(labels_test.values, dtype=torch.int64)

test_train_tensors = [features_train_tensor,features_test_tensor, labels_train_tensor, labels_test_tensor]
for tensor in test_train_tensors:
    print(str(tensor.shape))

torch.Size([200, 8])
torch.Size([50, 8])
torch.Size([200])
torch.Size([50])


In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(8, 16)
        self.fc2 = nn.Linear(16, 5)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = NeuralNetwork()


In [161]:
#Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.01)

In [163]:
num_epochs = 20
for epoch in range(num_epochs):
    optimizer = zero.grad()
    outputs = model(features_train_tensor)
    loss = criterion(outpus, labels_train_tensor)
    loss.backward
    optimizer.step()
    print(f'Epoch [{epoch}/{num_epochs}], Loss: {loss.item():.4f}')

TypeError: 'int' object is not iterable